In [56]:
import numpy
from numpy import *
import sympy
from sympy import *
import sympy as sym
init_printing(use_unicode=True)
from sympy.physics.quantum import TensorProduct as tp
from sympy.physics.quantum import Ket, Bra
import math

# Auxiliary functions

In [14]:
def inner_product(v,w):
    d = len(v); ip = 0
    for j in range(0,d):
        ip += conjugate(v[j])*w[j]
    return ip
#a,b,c,d = symbols("a b c d"); v = [b,a]; w = [c,d]; inner_product(v,w)

In [12]:
def norm(v):
    v = inner_product(v,v)
    return sqrt(v)
#v = [2,2]; norm(v)

In [ ]:
def cb(n,j):
    """
    Args:
        n (int): levels
        j (int): element that will be non-zero

    Returns:
        array: returns a standard base vector of C^n
    """
    vec = zeros(n,1)
    vec[j] = 1
    return vec

In [17]:
def proj(psi): 
    '''returns the projector in the psi vector'''
    d = psi.shape[0]
    P = zeros(d,d)
    for j in range(0,d):
        for k in range(0,d):
            P[j,k] = psi[j]*conjugate(psi[k])
    return P
#proj(cb(2,0))

In [13]:
def tsco(A):
    '''returns the conjugate transpose'''
    d = A.shape[0]
    new = zeros(d)
    new = sympy.transpose(A)
    new = sympy.conjugate(new)
    return new
#tsco(cb(2,0))

# Generic vector state

In [13]:
def psi_c(n, mtype=0):
    """
    Args:
        n (int): state vector levels
        mtype (int): array type, column array or row array. Zero (0) for
            row matrix and one (1) for row matrix

    Returns:
        array: Returns a row or column array depending on the 
               type (tp) selected with n levels. 
    """
    # Calculate the number of digits in n
    num_digits = len(str(n-1))
    # Create a list of symbolic variables with names 'c00' to 'cnn'
    psi = sympy.symbols(['c{:0{}}'.format(i, num_digits) for i in range(n)])
    # Create a numpy array of zeros with size n x 1
    A = zeros(n,1)
    # Set the first n elements of A to the psi symbols
    for j in range(0,n):
        A[j] = psi[j]
    # Return the resulting array A
    if mtype == 1:
        return transpose(A).as_mutable()
    return A

In [15]:
"""
Same situation as the function above, but write the coefficients 
with indices in binary form. Useful to verify some patterns after 
the actions of certain ports like CNOT among other utilities.
"""
def pbin(n, mtype=0):
    num_digits = len(str(n))
    A = zeros(n,1)
    psi = [sympy.symbols('c_{}'.format(format(i, '0{}b'.format(int(math.ceil(log(n, 2))))))) for i in range(n)]
    for j in range(0,n):
        A[j] = psi[j]
    if mtype == 1:
        return transpose(A).as_mutable()
    return A

In [38]:
"""
Interesting function to say exactly which matrix values
are different from zero and at the same time have a visualization
of the state vector as if it were written in Dirac notation
"""
def pbk(seq, dim=2, mtype=0):
    """
    Args:
        seq (str): input sequence. Ex: '010110'
        dim (int): computational basis dimension (default=2 for qubits)
        mtype (int): array type, column array or row array. Zero (default=0) for
            row matrix and one (1) for row matrix

    Returns:
        array: ket vector for the input sequence in the computational basis of dimension 'dim'
    """
    vec = []
    for digito in seq:
        vec.append(int(digito))
    n = len(vec)
    psi = cb(dim, vec[0])
    for j in range(1,n):
        psi = tp(psi,cb(dim, vec[j]))
    if mtype == 1:
        return transpose(psi).as_mutable()
    return psi

# Generic density matrix ($\rho$)

In [45]:
"""
Returns a symbolic generic rho
n = column dimension
Returns a symbolic generic rho
square matrix
n = row/column levels
"""
def rho_g(n, symbol=0):
    num_digits = len(str(n**2 - 1))
    A = zeros(n,n)
    l = 0
    if symbol == 1:
        rho = sympy.symbols(['sigma{:0{}}'.format(i, num_digits) for i in range(n**2)])
        for j in range(0,n):
            for k in range(0,n):
                    A[j,k] = rho[l]
                    l += 1
    else: 
        rho = sympy.symbols(['rho_{:0{}}'.format(i, num_digits) for i in range(n**2)])
        for j in range(0,n):
            for k in range(0,n):
                    A[j,k] = rho[l]
                    l += 1
    return A

In [48]:
"""
Returns a symbolic generic rho with indices in binary form
square matrix
n = row/column levels
"""
def rho_bin(n):
    rho = [sympy.symbols('rho_{}'.format(format(i, '0{}b'.format(int(math.ceil(log(n**2, 2))))))) for i in range(n**2)]
    A = zeros(n,n)
    l = 0
    for j in range(0,n):
        for k in range(0,n):
                A[j,k] = rho[l]
                l += 1
    return A

# Dirac notation

In [51]:
"""
Return Psi or rho in bra-ket notation
Sent Psi - return Psi in bra-ket notation
Send rho - return rho in bra-ket notation
"""
def mbk(matrix, dim=2):
    """
    Args:
        matrix (sympy.matrices.dense.MutableDenseMatrix): The input must be a sympy array.
                                            It can be a state vector or a density matrix

    Returns:
        Prints the matrix in Dirac notation
    """
    def convert_dim(x, dim, min_digits):
        """
        Helper function that checks how many digits the bra-ket will 
        have based on the dimension and size of the input matrix
        """
        digits = '0123456789'[:dim]
        result = ''
        while x > 0 or len(result) < min_digits:
            x, digit = divmod(x, dim)
            result = digits[digit] + result
        return result
    posicoes = []
    posicoes_bin = []
    val = []
    if isinstance(matrix, sympy.matrices.dense.MutableDenseMatrix):
        n_linhas, n_colunas = matrix.shape
        """
        If the row == 1 the notation will be Bra, if the column == 1 the notation will be Ket and if 
        it doesn't fit in any of these cases, then it's because it's a density matrix and will be handled by 'else'
        """
        if n_linhas == 1:
            Psi = 0
            x = len(matrix)
            for i in range(x):
                if matrix[i] != 0:
                    val.append(matrix[i])
                    posicoes.append(i)
                    posicoes_bin.append(convert_dim(i, dim, int(math.ceil(math.log(x)/math.log(dim)))))
            for i in range(len(val)):
                Psi = val[i] * Bra(posicoes_bin[i]) + Psi
            return simplify(Psi)
        if n_colunas == 1:
            Psi = 0
            x = len(matrix)
            for i in range(x):
                if matrix[i] != 0:
                    val.append(matrix[i])
                    posicoes.append(i)
                    posicoes_bin.append(convert_dim(i, dim, int(math.ceil(math.log(x)/math.log(dim)))))
            for i in range(len(val)):
                Psi = val[i] * Ket(posicoes_bin[i]) + Psi
            return Psi
        else:
            m, n = matrix.shape
            rho = 0
            for i in range(m):
                for j in range(n):
                    if matrix[i, j] != 0:
                        val.append(matrix[i, j])
                        posicoes.append((i, j))
                        posicoes_bin.append((convert_dim(i, dim, int(math.ceil(math.log(m)/math.log(dim)))),\
                                             convert_dim(j, dim, int(math.ceil(math.log(m)/math.log(dim))))))
            for i in range(len(val)):
                rho = val[i] * (Ket(posicoes_bin[i][0])) * (Bra(posicoes_bin[i][1])) + rho
            return rho

# A simpler way to put the expression result in a more elegant and eye-pleasing form

In [ ]:
def mysim(expr):
    return sympy.simplify(expr, rational=True)